In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd


In [6]:
# Load the dataset
data = pd.read_csv('dataset.csv')

# Select relevant columns
data = data[['Time', 'BatteryCurrent', 'BatteryVoltage']]


In [7]:
data.head()

,Time,BatteryCurrent,BatteryVoltage
0,0.0000,0.000194,3.972338
1,0.0015,0.048925,3.972173
2,0.0030,0.096904,3.972009
3,0.0045,0.144467,3.971847
4,0.0060,0.191261,3.971688


In [8]:
# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Prepare input features (Time, BatteryCurrent) and target variable (BatteryVoltage)
X = scaled_data[:, :-1]  # Time and BatteryCurrent as input
y = scaled_data[:, -1]   # BatteryVoltage as the target

In [9]:
# Create a sliding window of size n=5
def create_sequences(X, y, window_size=5):
    Xs, ys = [], []
    for i in range(len(X) - window_size):
        Xs.append(X[i:i + window_size])
        ys.append(y[i + window_size])
    return np.array(Xs), np.array(ys)


In [10]:

# Set window size and prepare the sequences
window_size = 5
X_seq, y_seq = create_sequences(X, y, window_size)

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [12]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))  # Output layer for BatteryVoltage prediction

In [13]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [17]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2,
         callbacks=[early_stopping_callback] )

Epoch 1/50
1200/1200 [==============================] - 6s 5ms/step - loss: 2.5278e-04 - val_loss: 2.5694e-04
Epoch 2/50
1200/1200 [==============================] - 9s 7ms/step - loss: 2.4439e-04 - val_loss: 2.7168e-04
Epoch 3/50
1200/1200 [==============================] - 10s 8ms/step - loss: 2.2867e-04 - val_loss: 2.3114e-04
Epoch 4/50
1200/1200 [==============================] - 10s 8ms/step - loss: 2.2766e-04 - val_loss: 1.9695e-04
Epoch 5/50
1200/1200 [==============================] - 10s 8ms/step - loss: 2.1656e-04 - val_loss: 2.5834e-04
Epoch 6/50
1200/1200 [==============================] - 9s 8ms/step - loss: 2.1423e-04 - val_loss: 2.5303e-04
Epoch 7/50
1200/1200 [==============================] - 10s 8ms/step - loss: 2.2666e-04 - val_loss: 2.4172e-04
Epoch 8/50
1200/1200 [==============================] - 10s 8ms/step - loss: 2.1357e-04 - val_loss: 2.2147e-04
Epoch 9/50
1200/1200 [==============================] - 10s 8ms/step - loss: 1.9685e-04 - val_loss: 2.2280e-04
Epoc

In [18]:
# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)


375/375 [==============================] - 1s 2ms/step - loss: 2.4981e-04
Test Loss: 0.0002498134854249656


In [22]:
# Function to predict next battery voltage given time and current
def predict_next_voltage(model, time_values, current_values, scaler, window_size=5):
    # Combine time and current values as input
    input_sequence = np.array([time_values, current_values]).T  # Shape (window_size, 2)
    
    # Create a temporary dataframe to scale only Time and BatteryCurrent
    input_sequence_with_dummy_voltage = np.zeros((window_size, 3))  # Create dummy column for BatteryVoltage
    input_sequence_with_dummy_voltage[:, :2] = input_sequence       # Fill Time and BatteryCurrent columns
    
    # Scale only the Time and BatteryCurrent columns
    input_sequence_scaled = scaler.transform(input_sequence_with_dummy_voltage)[:, :2]  # Scale and extract relevant columns
    
    # Reshape to match LSTM input shape (1, window_size, 2)
    input_sequence_scaled = np.reshape(input_sequence_scaled, (1, window_size, 2))
    
    # Predict the next battery voltage
    predicted_scaled_voltage = model.predict(input_sequence_scaled)
    
    # Inverse scale the prediction to get the actual battery voltage
    inverse_scaled_input = np.zeros((1, 3))  # Create dummy array for inverse scaling
    inverse_scaled_input[0, :2] = input_sequence[-1]  # Use the last Time and Current value
    inverse_scaled_input[0, 2] = predicted_scaled_voltage  # Add predicted BatteryVoltage
    
    # Inverse transform to get the original scale
    predicted_voltage = scaler.inverse_transform(inverse_scaled_input)[0][2]
    
    return predicted_voltage

In [23]:
# Example input (last 5 time steps and currents)
time_values = [0.0100, 0.0115, 0.0130, 0.0145, 0.0160]
current_values = [0.191261, 0.238776, 0.285627, 0.331660, 0.376706]

In [24]:
# Predict the next battery voltage
next_battery_voltage = predict_next_voltage(model, time_values, current_values, scaler)
print("Predicted Next Battery Voltage:", next_battery_voltage)


d:\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 274ms/step
Predicted Next Battery Voltage: 2.6264595878081933


C:\Users\saura\AppData\Local\Temp\ipykernel_2056\4209454503.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  inverse_scaled_input[0, 2] = predicted_scaled_voltage  # Add predicted BatteryVoltage
